In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
from resnet18 import ResNet, BasicBlock
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import cv2
import pickle

%matplotlib inline

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_resnet18_model(model_path, num_classes=2, num_channels=3):
    """

    :param model_path: Path to ResNet18 model
    :param num_channels: Number of channels in image, also depends on trained model
    :return: Instance of the model (torchvision.models.resnet18() in eval mode)
    """

    # Create an instance of the ResNet18 model
    model = ResNet(img_channels=num_channels, num_layers=18, block=BasicBlock, num_classes=num_classes).to(device)

    # Load the saved state dictionary into the model
    # map_location = device makes code compatible for CPU and GPU
    model.load_state_dict(torch.load(model_path, map_location=device))

    # Set the model to evaluation mode
    model.eval()

    return model


def load_image_with_transform(input_img, num_channels=3):
    input_img = input_img.copy()

    # input_img.resize((224, 224, 3))
    # input_img = input_img * 255 / np.max(input_img)
    # input_img = input_img.astype(np.uint8)

    input_img = Image.fromarray(input_img)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
        # transforms.Grayscale(num_output_channels=1)
    ])

    input_img = transform(input_img)

    n = 224

    input_img = torch.tensor(input_img.clone().detach().requires_grad_(True), dtype=torch.float)
    input_img = input_img.view(1, num_channels, n, n).to(device)

    return input_img

In [3]:
with open('/home/raja/Desktop/MICCAI/code/18_resnet/models/best.pkl', 'rb') as f:
    model = pickle.load(f)


In [4]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
img = plt.imread('data/NONLYMPHOCYTES/img_2_cropped_30.jpg')
transformed_img =  load_image_with_transform(img, num_channels=3)

outputs = model(transformed_img)
outputs = F.softmax(outputs, dim=1)
predicted_class = torch.argmax(outputs, dim=1)
confidence_scores = torch.nn.functional.softmax(outputs, dim=1)

print(outputs)

_, preds = torch.max(outputs.data, 1)

#classes = ['Lymphocyte', 'Non-Lymphocyte']
classes = [0,1]

class_prediction = classes[preds.tolist()[0]]

class_prediction

/tmp/ipykernel_8043/499375677.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_img = torch.tensor(input_img.clone().detach().requires_grad_(True), dtype=torch.float)


tensor([[5.7015e-05, 9.9994e-01]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


1

In [8]:
class_predictions = []

for img_name in os.listdir('/home/raja/Desktop/segment-anything/test-data-output/test-output-v2'):
    img = plt.imread(os.path.join('/home/raja/Desktop/segment-anything/test-data-output', 'test-output-v2', img_name))
    transformed_img = load_image_with_transform(img, num_channels=3)

    outputs = model(transformed_img)
    outputs = F.softmax(outputs, dim=1)

    _, preds = torch.max(outputs.data, 1)

    classes = [0, 1]
    class_prediction = classes[preds.tolist()[0]]

    class_predictions.append(class_prediction)

#print(class_predictions)


/tmp/ipykernel_8043/499375677.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_img = torch.tensor(input_img.clone().detach().requires_grad_(True), dtype=torch.float)


In [9]:
import csv
import os

# Combine image names and predicted labels
data = zip(os.listdir('/home/raja/Desktop/segment-anything/test-data-output/test-output-v2'), class_predictions)

# Write to CSV file
with open('/home/raja/Desktop/MICCAI/code/18_resnet/predictions-resnet.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Image Name', 'Label'])
    for row in data:
        writer.writerow(row)


In [10]:
import csv
import re

# Define input and output file names
input_file = '/home/raja/Desktop/MICCAI/code/18_resnet/predictions-resnet.csv'
output_file = '/home/raja/Desktop/MICCAI/code/18_resnet/output-resnet.csv'

# Define dictionary to store counts for each image
image_counts = {}

# Define regular expression to extract image number from filename
img_regex = re.compile(r'img_(\d+)_cropped_\d+.jpg')

# Read input file and count labels for each image
with open(input_file, 'r') as f:
    reader = csv.reader(f)
    next(reader)  # skip header row
    for row in reader:
        image_name, label = row
        img_match = img_regex.match(image_name)
        if img_match:
            image_number = int(img_match.group(1))
            if image_number in image_counts:
                if label == '0':
                    image_counts[image_number][0] += 1
                else:
                    image_counts[image_number][1] += 1
            else:
                if label == '0':
                    image_counts[image_number] = {0: 1, 1: 0}
                else:
                    image_counts[image_number] = {0: 0, 1: 1}

# Write output file with counts for each image
with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Image', 'Predicted-Lymphocyte'])
    for image_number, counts in sorted(image_counts.items()):
        writer.writerow(['img_{}'.format(image_number), counts[0]])


In [11]:
import pandas as pd

# read the csv file into a pandas DataFrame
df = pd.read_csv('/home/raja/Desktop/MICCAI/code/18_resnet/output-resnet.csv')

# create a new DataFrame with the full range of image numbers
img_range = range(20000)
new_df = pd.DataFrame({'Image': ['img_' + str(i) for i in img_range], 'Predicted-Lymphocyte': 0})

# merge the new DataFrame with the original DataFrame using a left join
merged_df = pd.merge(new_df, df, on='Image', how='left')


# write the merged DataFrame to a new csv file
merged_df.to_csv('/home/raja/Desktop/MICCAI/code/18_resnet/new_file-itr2.csv', index=False)

# Load the Excel file
df = pd.read_csv('/home/raja/Desktop/MICCAI/code/18_resnet/new_file-itr2.csv')

# Fill the 'predicted' column with 0 for NA values
df['Predicted-Lymphocyte_y'].fillna(0, inplace=True)

# Save the modified DataFrame back to Excel
df.to_csv('/home/raja/Desktop/MICCAI/code/18_resnet/new_file-itr2.csv', index=False)


In [12]:
'''
import pandas as pd

# read the input CSV file into a pandas dataframe
df = pd.read_csv('/home/raja/Desktop/MICCAI/code/18_resnet/new_file.csv')

# define the function to assign class labels based on ground truth values
def assign_class(gt):
    if gt == 0:
        return '0 (None)'
    elif 1 <= gt <= 5:
        return '1-5'
    elif 6 <= gt <= 10:
        return '6-10'
    elif 11 <= gt <= 20:
        return '11-20'
    elif 21 <= gt <= 50:
        return '21-50'
    elif 51 <= gt <= 200:
        return '51-200'
    else:
        return '>200'

# assign class labels to ground truth values
df['Ground-truth Class'] = df['Ground-truth'].apply(assign_class)

# assign class labels to predicted values based on the constraints
df['Predicted Class'] = df.apply(lambda row: assign_class(row['Predicted-Lymphocyte_y']), axis=1)

# calculate the confusion matrix
confusion_matrix = pd.crosstab(df['Ground-truth Class'], df['Predicted Class'])

# define the desired order of the classes
desired_order = ['0 (None)', '1-5', '6-10', '11-20', '21-50']

# reindex the confusion matrix to match the desired order
confusion_matrix = confusion_matrix.reindex(index=desired_order, columns=desired_order, fill_value=0)

# print the confusion matrix
print(confusion_matrix)

'''

"\nimport pandas as pd\n\n# read the input CSV file into a pandas dataframe\ndf = pd.read_csv('/home/raja/Desktop/MICCAI/code/18_resnet/new_file.csv')\n\n# define the function to assign class labels based on ground truth values\ndef assign_class(gt):\n    if gt == 0:\n        return '0 (None)'\n    elif 1 <= gt <= 5:\n        return '1-5'\n    elif 6 <= gt <= 10:\n        return '6-10'\n    elif 11 <= gt <= 20:\n        return '11-20'\n    elif 21 <= gt <= 50:\n        return '21-50'\n    elif 51 <= gt <= 200:\n        return '51-200'\n    else:\n        return '>200'\n\n# assign class labels to ground truth values\ndf['Ground-truth Class'] = df['Ground-truth'].apply(assign_class)\n\n# assign class labels to predicted values based on the constraints\ndf['Predicted Class'] = df.apply(lambda row: assign_class(row['Predicted-Lymphocyte_y']), axis=1)\n\n# calculate the confusion matrix\nconfusion_matrix = pd.crosstab(df['Ground-truth Class'], df['Predicted Class'])\n\n# define the desired 